<a href="https://colab.research.google.com/github/mansoorhamidzadeh/transformers/blob/main/persianQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Persian QA
you can test the result in [huggingface](https://huggingface.co/mansoorhamidzadeh/parsbert-persian-QA)

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("question-answering", model="mansoorhamidzadeh/parsbert-persian-QA")
context = "متن زمینه که شامل اطلاعات مرتبط با سوال شما است."
question = "سوال شما چیست؟"
result = pipe(question=question, context=context)

print(f"Answer: {result['answer']}")

In [ ]:

!pip install transformers
!pip install accelerate -U

In [ ]:

!pip install datasets

In [ ]:
import transformers
model_checkpoint = "HooshvareLab/bert-fa-zwnj-base"
batch_size = 1

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/292 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/426k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/134 [00:00<?, ?B/s]

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from datasets import load_dataset
datasets = load_dataset("SajjadAyoubi/persian_qa")

Generating train split:   0%|          | 0/9008 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/930 [00:00<?, ? examples/s]

In [ ]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 9008
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 930
    })
})

In [ ]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset)
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [ ]:
show_random_elements(datasets["train"])

,id,title,context,question,answers
0,7152,جن,جن در باورهای عامیانه موجودی «فراطبیعی» در اساطیر اسلامی است که به باور باورمندان جسمی از آتش یا بخار است که عقل و هوش دارد، توسط انسان دیده نمی‌شود، می‌تواند در اشکال مختلف ظاهر شود، کارهای سنگین انجام دهد و از آتش بدون دود خلق شده‌است. به گفته قرآن، محمد به عنوان پیامبر نزد جن و انس فرستاده شد و همچنین اجنه در کنار انسان‌ها و فرشتگان سه گروه مخلوقات هوشمند هستند. رابطه جن‌ها با ابلیس روشن نیست؛ قرآن یک مرتبه ابلیس را یک جن و در جای دیگر یک فرشته نامیده‌است و همین سبب سردرگمی‌های بسیار شده‌است. اعتقاد به جن نزد اعراب پیش از اسلام نیز وجود داشته‌است. منشأ دقیق اعتقاد به جن به روشنی مشخص نیست. برخی از خاورمیانه‌شناسان معتقدند که اجنه در نظر نخستین باورمندان به آن‌ها، ارواح بدخواه ساکن بیابان‌ها و مکان‌های کثیف بودند که اغلب به شکل حیوانات درمی‌آمدند. برخی دیگر معتقدند آن‌ها در اصل خدایان طبیعت بودند که به تدریج با افزایش اهمیت دیگر خدایان به حاشیه رانده شدند. با این وجود، بسیاری از اعراب در دوران پیش از اسلام جن‌ها را پرستش می‌کردند اما معتقد بودند اینان برخلاف خدایان، جاودان نیستند. هر چند فانی بودن اجنه سبب شده بود در مرتبه‌ای پایین‌تر از خدایان قرار داده شوند، به نظر می‌رسد آن‌ها در زندگی روزمره اعراب قبل از اسلام اهمیت بیشتری نسبت به خدایان داشتند. طبق عقاید رایج میان اعراب اجنه به فال‌گویان، فلاسفه پیش از اسلام و شاعران الهام می‌کردند.,جن ها به لحاظ تفکر و تعقل چه گونه هستند؟,"{'text': ['عقل و هوش دارد'], 'answer_start': [109]}"
1,6800,حافظ,خواجه شمسُ‌الدّینْ محمّدِ بن بهاءُالدّینْ محمّدْ حافظِ شیرازی (زادهٔ ۷۲۷ هجری قمری (معادل ۱۳۱۵ میلادی) – درگذشتهٔ ۷۹۲ هجری قمری (معادل ۱۳۹۰ میلادی) در شیراز)، مشهور به لِسانُ‌الْغِیْب، تَرجُمانُ الْاَسرار، لِسانُ‌الْعُرَفا و ناظِمُ‌الاُولیاء، شاعر فارسی‌زبانِ سدهٔ هشتم ه‍.ق شیراز است. بیش‌تر شعرهای او غزل است. مشهور است که حافظ به شیوهٔ سخن‌پردازی خواجوی کرمانی گرویده و همانندیِ سخنش با شعرِ خواجو مشهور است. حافظ از مهم‌ترین اثرگذاران بر شاعرانِ فارسی‌زبانِ پس از خود شناخته می‌شود. در سده‌های هجدهم و نوزدهم میلادی اشعار او به زبان‌های اروپایی نیز ترجمه شد و نامش به محافل ادبی جهان غرب نیز راه یافت. هر سال در ۲۰ مهر مراسم بزرگ‌داشت حافظ در آرامگاهش در شیراز با حضور پژوهشگرانی از اقصا نقاط دنیا برگزار می‌شود. در تقویم رسمی ایران ۲۰ مهر روز بزرگداشت حافظ نامیده شده‌است. مشهور است که حافظ در سال‌های نوجوانی و آغاز جوانی شاگرد نانوا بوده و از نسخه‌های خطّی نیز رونوشت بر می‌داشته ‌است. آگاهی‌ها دربارهٔ تحصیل و تدریس حافظ فقط از اشاره‌های خودش در شعرها و نیز مقدّمه‌ای است که محمّد گلندام در نهایت اختصار نگاشته‌است.,حافظ در پیری و کهنسالی به چه کاری مشغول بود؟,"{'text': [], 'answer_start': []}"
2,3446,نسلیهان آتاگول,نسلیهان آتاگول (به ترکی استانبولی: Neslihan Atagül) (زادهٔ ۲۰ اوت ۱۹۹۲) بازیگر اهل ترکیه است. او با بازی در فیلم سینمایی عشق اول (۲۰۰۶) به شهرت رسید. او اولین بازیگری خود را در مجموعه تلویزیونی برگ‌ریزان (۱۰–۲۰۰۶) انجام داد و بعداً در فیلم برزخ (۲۰۱۱) بازی کرد. در همان سال او در مجموعه تلویزیونی پدر عزیزم (۲۰۱۱) ظاهر شد. آتاگول برای بازی در فیلم‌های فاتح حربیه (۲۰۱۳–۲۰۱۴) و عشق سیاه (۲۰۱۵–۲۰۱۷) مورد توجه بین‌المللی قرار گرفت. او مورد تحسین منتقدان قرار گرفت و موفق به کسب افتخارات زیادی شد.نسلیهان آتاگول در تاریخ ۲۰ اوت ۱۹۹۲ در استانبول متولد شد. پدرش راننده و مادرش یک زن خانه‌دار بود. پدرش از چرکس و مادرش بلاروسی تبار است. نسلیهان در اکتبر ۲۰۱۳، شروع به دوستی با ستارهٔ فاتیح هاربیه، کادیر دوغلو کرد. آنها در ۲۵ نوامبر ۲۰۱۵ نامزد شدند و در ژوئیه ۲۰۱۶ ازدواج کردند. در همان سال، پدرش را از دست داد و همسایگانش ادعا کردند که علاقه‌ای به او ندارد.,نسلیهان اتاگول در چه سالی در فیلم عشق اول بازی کرد؟,"{'text': ['۲۰۰۶'], 'answer_start': [130]}"
3,2766,رود آیلند,رودآیلند (به انگلیسی: Rhode Island) ایالتی است در شمال شرقی آمریکا و یکی از ایالت‌های نیوانگلند است. این ایالت از سمت شمال و شرق با ایالت ماساچوست، از سمت غرب با ایالت کنکتیکت و از سمت جنوب با اقیانوس اطلس مرز مشترک دارد. مساحت تقریبی ایالت رود آیلند در حدود سه هزار و یکصد و پنجاه هزار کیلومترمربع و جمعیتی بالغ بر یک میلیون و پنجاه هزار نفر دارد. پایتخت و بزرگترین شهر این ایالت، پراویدنس نام دارد. رودآیلند یکی از سیزده ایالتی است که در جریان انقلاب آمریکا علیه سلطه بریتانیا قیام کردند. این ا

In [ ]:
max_length = 384
doc_stride = 128

In [ ]:
pad_on_right = tokenizer.padding_side == "right"

In [ ]:
def prepare_train_features(examples):
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples.pop("offset_mapping")

    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        sequence_ids = tokenized_examples.sequence_ids(i)
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:

                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [ ]:
tokenized_datasets = datasets.map(prepare_train_features, batched=True, remove_columns=datasets["train"].column_names)


Map:   0%|          | 0/9008 [00:00<?, ? examples/s]

Map:   0%|          | 0/930 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

pytorch_model.bin:   0%|          | 0.00/473M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at HooshvareLab/bert-fa-zwnj-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(42000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elem

In [ ]:
torch.cuda.is_available()

True

In [ ]:

!pip install accelerate -U

In [ ]:
model_name = "parsbert-persian-QA"
args = TrainingArguments(
    model_name,
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    push_to_hub=True,
    save_steps=5000
)

In [ ]:
from transformers import default_data_collator

data_collator = default_data_collator

In [ ]:
trainer = Trainer(
    model.to(device),
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [ ]:


trainer.train()

Epoch,Training Loss,Validation Loss
1,2.949200,2.644243
2,2.351500,2.556234
3,1.579100,3.598848


In [ ]:
trainer.train("")

Loading model from C:/Users/marzi/Downloads/parsbert-finetuned-persianQA/checkpoint-20000).
***** Running training *****
  Num examples = 9205
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 27615
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 2
  Continuing training from global step 20000
  Will skip the first 2 epochs then the first 1590 batches in the first epoch. If this takes a lot of time, you can add the `--ignore_data_skip` flag to your launch command, but you will resume the training on data already seen by your model.


  0%|          | 0/1590 [00:00<?, ?it/s]

Epoch,Training Loss,Validation Loss
3,1.470300,3.428255


Saving model checkpoint to parsbert-finetuned-persianQA\checkpoint-25000
Configuration saved in parsbert-finetuned-persianQA\checkpoint-25000\config.json
Model weights saved in parsbert-finetuned-persianQA\checkpoint-25000\pytorch_model.bin
tokenizer config file saved in parsbert-finetuned-persianQA\checkpoint-25000\tokenizer_config.json
Special tokens file saved in parsbert-finetuned-persianQA\checkpoint-25000\special_tokens_map.json
tokenizer config file saved in parsbert-finetuned-persianQA\tokenizer_config.json
Special tokens file saved in parsbert-finetuned-persianQA\special_tokens_map.json
***** Running Evaluation *****
  Num examples = 930
  Batch size = 1


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=27615, training_loss=0.3851581684687896, metrics={'train_runtime': 7296.5564, 'train_samples_per_second': 3.785, 'train_steps_per_second': 3.785, 'total_flos': 5411782452948480.0, 'train_loss': 0.3851581684687896, 'epoch': 3.0})